In [1]:
%%capture
from numpy import int16
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime
from threading import Timer
from pandas import read_excel

from oactf import actif, patient, driver

date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [2]:
def the_upload():
    schooling_dreams = read_excel(f"~/Downloads/schooling_dreams {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
    schooling_dreams = schooling_dreams[schooling_dreams.closed == False]
    payed_schooling_dreams = schooling_dreams[schooling_dreams.eskew_peye == "1"]
    return schooling_dreams, payed_schooling_dreams

t = Timer(1000.0,the_upload)
t.start()
schooling_dreams,payed_schooling_dreams = the_upload()

In [3]:
driver.quit()

In [4]:
schooling_dreams.closed.unique()
schooling_dreams.eskew_peye.unique()
payed_schooling_dreams.eskew_peye.unique()

array([False])

array(['1', '---'], dtype=object)

array(['1'], dtype=object)

In [5]:
actif.id_patient.count()
patient.id_patient.count()
schooling_dreams.caseid.count()
payed_schooling_dreams.caseid.count()

18815

165880

162

153

In [6]:
actif.to_excel('actif.xlsx',index=False)

In [7]:
comp = actif[actif.complete_at_least=="yes"]
comp.id_patient.count()
comp.test_results.fillna("0,",inplace=True)
comp.test_results.value_counts()

17538

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_50839/319073030.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp.test_results.fillna("0,",inplace=True)


0,          16000
0,,2,         870
2,            658
0,,3,           4
1,              2
0,,2,,3,        1
0,,1,           1
2,,3,           1
3,              1
Name: test_results, dtype: int64

In [8]:
ovc_data= comp
neg_data = comp[
    (comp.test_results!="0,,3,")&
    (comp.test_results!="0,,1,,2,")&
    (comp.test_results!="0,,2,,3,")&
    (comp.test_results!="3,")&
    (comp.test_results!="2,,3,")
]
ovc_data.id_patient.count()
neg_data.id_patient.count()
neg_data.test_results.value_counts()

17538

17531

0,       16000
0,,2,      870
2,         658
1,           2
0,,1,        1
Name: test_results, dtype: int64

In [9]:
positive_dreams= ovc_data[
    (ovc_data.test_results=="1,")|
    (ovc_data.test_results=="0,,1,")
]
positive_dreams.id_patient.count()

3

In [10]:
positive_dreams

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,ps_1519,ps_2024,secondary_1014,secondary_1519,secondary_2024,complete_1014,complete_1519,complete_2024,complete_at_least,isEnrolledQ4
3076,151637,NaN,Artibonite,Grande Saline,21,0,yes,yes,1,1,...,no,primary,no,no,secondary,no,no,no,yes,no
3965,162212,NaN,Nord,Cap-Haïtien,21,0,yes,yes,1,1,...,primary,no,no,secondary,no,no,no,no,yes,no
13041,85541,NaN,Ouest,Delmas,21,0,yes,yes,2,0,...,no,primary,no,no,no,no,no,no,yes,no


In [11]:
ovc_data.ovc_age.unique()
ovc_data.parenting.unique()
ovc_data.parenting_detailed.unique()

array(['18-24', '15-17', '25-29', '10-14', 'not_valid_age'], dtype=object)

array(['no', 'yes'], dtype=object)

array(['no', 'yes', 'has_started'], dtype=object)

In [12]:
ovcS_dreams = ovc_data[
    (ovc_data.ovc_age=="10-14")|
    (ovc_data.ovc_age=="15-17")
]
ovcS_dreams['Gender'] = "Female"
ovcS_dreams.id_patient.count()

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_50839/39440959.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ovcS_dreams['Gender'] = "Female"


11306

In [13]:
ovcS_dreams = patient.merge(ovcS_dreams,on="id_patient",how="right")


In [14]:
ovcS_dreams[ovcS_dreams.dreams_code.isin(payed_schooling_dreams.dreams_code)].id_patient.count()
ovcS_dreams = ovcS_dreams[~ovcS_dreams.dreams_code.isin(payed_schooling_dreams.dreams_code)]
ovcS_dreams.id_patient.count()

135

11171

In [15]:
ovcS_dreams.parenting_detailed.unique()

array(['no', 'yes', 'has_started'], dtype=object)

In [16]:
datim_ovc_dreams = ovcS_dreams.pivot_table(index="commune",values="id_patient",columns=["Gender","ovc_age"],aggfunc='count',fill_value=0)
datim_ovc_dreams

Gender                         Female      
ovc_age                         10-14 15-17
commune                                    
Cap-Haïtien                       343   225
Carrefour                         367   271
Delmas                             75   128
Desdunes                          264   187
Dessalines                        755   190
Grande Saline                     436   218
Gressier                          180   145
Kenscoff                          994   693
La Chapelle                       224   162
Liancourt                         149   115
Limonade                          132   108
Milot                              46    42
Montrouis                         119   107
Petite Rivière de l'Artibonite    285   187
Plaine-du-Nord                    100    67
Port-au-Prince                    320   331
Pétionville                       231   207
Quartier-Morin                     59    45
Saint-Marc                        590   546
Tabarre                           174   148
Verrettes                         748   458

In [17]:
datim_ovc_dreams[('Female','<1')] = 0
datim_ovc_dreams[('Female','1-4')] = 0
datim_ovc_dreams[('Female','5-9')] = 0
datim_ovc_dreams[('Female','18+')] = 0
datim_ovc_dreams[('Male','<1')] = 0
datim_ovc_dreams[('Male','1-4')] = 0
datim_ovc_dreams[('Male','5-9')] = 0
datim_ovc_dreams[('Male','10-14')] = 0
datim_ovc_dreams[('Male','15-17')] = 0
datim_ovc_dreams[('Male','18+')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),  
    ('Male','<1'),
    ('Male','1-4'),
    ('Male','5-9'),
    ('Male','10-14'),
    ('Male','15-17'),   
    ('Male','18+'),   
]

datim_ovc_dreams = datim_ovc_dreams.reindex(columns= columns_orientation)
datim_ovc_dreams

Gender                         Female                         Male          \
ovc_age                            <1 1-4 5-9 10-14 15-17 18+   <1 1-4 5-9   
commune                                                                      
Cap-Haïtien                         0   0   0   343   225   0    0   0   0   
Carrefour                           0   0   0   367   271   0    0   0   0   
Delmas                              0   0   0    75   128   0    0   0   0   
Desdunes                            0   0   0   264   187   0    0   0   0   
Dessalines                          0   0   0   755   190   0    0   0   0   
Grande Saline                       0   0   0   436   218   0    0   0   0   
Gressier                            0   0   0   180   145   0    0   0   0   
Kenscoff                            0   0   0   994   693   0    0   0   0   
La Chapelle                         0   0   0   224   162   0    0   0   0   
Liancourt                           0   0   0   149   115   0    0   0   0   
Limonade                            0   0   0   132   108   0    0   0   0   
Milot                               0   0   0    46    42   0    0   0   0   
Montrouis                           0   0   0   119   107   0    0   0   0   
Petite Rivière de l'Artibonite      0   0   0   285   187   0    0   0   0   
Plaine-du-Nord                      0   0   0   100    67   0    0   0   0   
Port-au-Prince                      0   0   0   320   331   0    0   0   0   
Pétionville                         0   0   0   231   207   0    0   0   0   
Quartier-Morin                      0   0   0    59    45   0    0   0   0   
Saint-Marc                          0   0   0   590   546   0    0   0   0   
Tabarre                             0   0   0   174   148   0    0   0   0   
Verrettes                           0   0   0   748   458   0    0   0   0   

Gender                                          
ovc_age                        10-14 15-17 18+  
commune                                         
Cap-Haïtien                        0     0   0  
Carrefour                          0     0   0  
Delmas                             0     0   0  
Desdunes                           0     0   0  
Dessalines                         0     0   0  
Grande Saline                      0     0   0  
Gressier                           0     0   0  
Kenscoff                           0     0   0  
La Chapelle                        0     0   0  
Liancourt                          0     0   0  
Limonade                           0     0   0  
Milot                              0     0   0  
Montrouis                          0     0   0  
Petite Rivière de l'Artibonite     0     0   0  
Plaine-du-Nord                     0     0   0  
Port-au-Prince                     0     0   0  
Pétionville                        0     0   0  
Quartier-Morin                     0     0   0  
Saint-Marc                         0     0   0  
Tabarre                            0     0   0  
Verrettes                          0     0   0

In [18]:
#positive_dreams.to_excel('./ovc_results/positive_data/positive_dreams.xlsx',na_rep="",index=False)
#datim_ovc_dreams.to_excel('./ovc_results/datim_ovc_dreams.xlsx',na_rep="",index=True)

ovcS_dreams.to_excel(f"./ovc_results/valid_ovc_data_{date_du_jour}.xlsx",na_rep="",index=False)
positive_dreams.to_excel(f"./ovc_results/positive_data/positive_dreams_{date_du_jour}.xlsx",na_rep="",index=False)
datim_ovc_dreams.to_excel(f"./ovc_results/datim_ovc_dreams_{date_du_jour}.xlsx",na_rep="",index=True)

